In [ ]:
#See README file to see where code is sourced from.

In [ ]:
import cv2
import numpy as np
import os
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
import keras
from keras.models import Sequential, Model,load_model
#from keras.optimizers import SGD
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,MaxPool2D, Dropout
from keras.preprocessing import image
from keras.initializers import glorot_uniform
from tensorflow.keras.applications.resnet50 import preprocess_input
import pandas as pd
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report
from imblearn.over_sampling import RandomOverSampler
import scipy
import random
import gc
import glob
from keras.utils.vis_utils import plot_model


In [ ]:
#Read in CSV file for FER2013 data to be manipulated.
data = pd.read_csv('/com.docker.devenvironments.code/fer2013/icml_face_data.csv')

In [ ]:
#Define which CSV data is image data and which is label data.
pixel_data = data[' pixels']
label_data = data['emotion']

In [ ]:
#Print number of images that can be generated from CSV data. 
len(label_data)

In [ ]:
#Pre-process the CSV data and reshape it into a 48x48 image.
def preprocess_pixels(pixel_data):
  images = []
  for i in range(len(pixel_data)):
    img = np.fromstring(pixel_data[i], dtype='int', sep=' ')
    img = img.reshape(48,48,1)
    images.append(img)

  X = np.array(images)
  return X



In [ ]:
#Duplicate data for emotion classes with lower samples, 'auto' means all will have the same number of examples.
oversampler = RandomOverSampler(sampling_strategy='auto')

X_over, Y_over = oversampler.fit_resample(pixel_data.values.reshape(-1,1), label_data)

In [ ]:
#Test that the reshaped data fits a 48x48 image.
X_over_series = pd.Series(X_over.flatten())
X_over_series

In [ ]:
#Preprocess data and turn into image form.
X = preprocess_pixels(X_over_series)
Y = Y_over

In [ ]:
Y = Y_over.values.reshape(Y.shape[0],1)
Y.shape

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.1, random_state = 45)
print(X_train.shape, Y_train.shape)

In [ ]:
#Test random image from dataset and display it to check that it has formed an image.
plt.imshow(X[25000,:,:,0])

In [ ]:
#Identity Block

def identity(X, f, filters, stage, block):
    conv_name_base='res' + str(stage) + block + '_branch'
    bn_name_base='bn' + str(stage) + block + '_branch'
    F1, F2, F3 = filters
    
    X_shortcut = X

    X = Conv2D(filters=F2, kernel_size=(1, 1), strides=(1, 1), padding='same', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    X = Add()([X, X_shortcut])# SKIP Connection
    X = Activation('relu')(X)

    return X

In [ ]:
#Convolutional Block

def convolutional (X, f, filters, stage, block, s=2):
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    F1, F2, F3 = filters

    X_shortcut = X

    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '1', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base + '1')(X_shortcut)

    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

In [ ]:
#ResNet50 Block

def ResNet50(input_shape=(48,48,3)):

    X_input = Input(input_shape)

    X = ZeroPadding2D((3, 3))(X_input)

    X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    X = convolutional(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    X = identity(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity(X, 3, [64, 64, 256], stage=2, block='c')


    X = convolutional(X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
    X = identity(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity(X, 3, [128, 128, 512], stage=3, block='d')

    X = convolutional(X, f=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
    X = identity(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity(X, 3, [256, 256, 1024], stage=4, block='f')

    X = X = convolutional(X, f=3, filters=[512, 512, 2048], stage=5, block='a', s=2)
    X = identity(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity(X, 3, [512, 512, 2048], stage=5, block='c')

    X = AveragePooling2D(pool_size=(2, 2), padding='same')(X)
    
    model = Model(inputs=X_input, outputs=X, name='ResNet50')

    return model


In [ ]:
resnet = ResNet50(input_shape=(48,48,1))

In [ ]:
#Transfer Learning to bypass most training since will take days.

newModel = resnet.output
newModel = Flatten()(newModel)
newModel = Dense(256, activation='relu', name='fc1',kernel_initializer=glorot_uniform(seed=0))(newModel)
newModel = Dense(128, activation='relu', name='fc2',kernel_initializer=glorot_uniform(seed=0))(newModel)
newModel = Dense(7,activation='softmax', name='fc3',kernel_initializer=glorot_uniform(seed=0))(newModel)


In [ ]:
#Feed input data into model, output into the fully connected layers.

model = Model(inputs=resnet.input, outputs=newModel)


In [ ]:
#Load weights from training, define learning rate and compile model.
adam = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
#Summary of model's layers and parameters.
model.summary()

In [ ]:
#Hot encode the emotion labels before training.
y_train = to_categorical(Y_train, num_classes=7)
y_test = to_categorical(Y_test, num_classes=7)


In [ ]:
#Define what each result from the prediction function means for each emotion.
label_dict = {0 : 'Angry', 1 : 'Disgust', 2 : 'Fear', 3 : 'Happiness', 4 : 'Sad', 5 : 'Surprise', 6 : 'Neutral'}
#Prediction function, returns the vector of image label, as well as index of image location.
preds = model.predict(X_train)
def get_class(preds):
  pred_class = np.zeros((preds.shape[0],1))

  for i in range(len(preds)):
   pred_class[i] = np.argmax(preds[i])

  return pred_class

pred_class_train = get_class(preds)

In [ ]:
#Train Model using Train data.

#es=EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=20)
#mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max')

#Model_Result = model.fit(X_train, y_train, validation_data=(X_test,y_test),epochs=30,steps_per_epoch=1000,verbose=1,callbacks=[mc,es])
#model.save('ResNet-HardCode')


In [ ]:
#Load the saved model, this includes the training weights.
model = load_model('ResNet-HardCode')
model.summary()

In [ ]:
#Pick a random image from the RandomImages folder, containing all test images from FER2013, not from CSV data.
test_image_count = len(os.listdir('/com.docker.devenvironments.code/fer2013/RandomImages'))
print('There are ' + str(test_image_count) + ' images in the testing folder')

test_image = random.choice(os.listdir('/com.docker.devenvironments.code/fer2013/RandomImages'))

img_path = ('/com.docker.devenvironments.code/fer2013/RandomImages/' + test_image)
#Load the selected image
img = image.load_img(img_path, grayscale=True, target_size=(48,48,1))


#Translate image into array form so it can be fed into the prediction function.
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

prediction = np.argmax(model.predict(x))

#Display the image and the predicted emotion as an output.
print('The predicted emotion is : ' + label_dict[prediction])
my_image = image.load_img(img_path)
plt.imshow(my_image)

In [ ]:
#Evaluate on Test Data:
results = model.evaluate(X_test, y_test, batch_size = 256)
print("test loss, test acc",results)

In [ ]:
#Plot graphs for accuracy and loss, can ONLY be done in the same execution as training since Model_Result doesnt save.
#plt.subplot(2,1,1)
#plt.plot(Model_Result.history['accuracy'])
#plt.plot(Model_Result.history['val_accuracy'])
#plt.title('Manual ResNet CSV Model Original - Accuracy')
#plt.ylabel('accuracy')
#plt.xlabel('epoch')
#plt.legend(['train', 'test'], loc = 'lower right')


In [ ]:
#plt.subplot(2,1,2)
#plt.plot(Model_Result.history['loss'])
#plt.plot(Model_Result.history['val_loss'])
#plt.title('Manual ResNet CSV Model Original - Loss')
#plt.ylabel('loss')
#plt.xlabel('epoch')
#plt.legend(['train', 'test'], loc = 'upper right')
